In [38]:
!pip install streamlit

In [39]:
!pip install pyngrok

In [40]:
!unzip '/content/drive/MyDrive/Colab Notebooks/encore/csv/archive.zip'

Archive:  /content/drive/MyDrive/Colab Notebooks/encore/csv/archive.zip
replace E Commerce Dataset.xlsx? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: E Commerce Dataset.xlsx  


In [41]:
%%writefile app.py
import streamlit as st
import pandas as pd
import joblib
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
import plotly.graph_objects as go

@st.cache_data
def load_customer_data():
    df = pd.read_excel('/content/E Commerce Dataset.xlsx', sheet_name=1)
    return df

@st.cache_resource
def load_model():
    return joblib.load('/content/drive/MyDrive/Colab Notebooks/encore/project02/best_model.pkl')

def preprocess_data(df):
    # 결측치 처리
    df['Tenure'].fillna(np.random.randint(61,72), inplace=True)
    df['WarehouseToHome'].fillna(0, inplace=True)
    df['CouponUsed'].fillna(0, inplace=True)
    df['OrderCount'].fillna(0, inplace=True)

    # 결측치가 있는 행 제거
    df = df[df['HourSpendOnApp'].notna()]
    df = df[df['DaySinceLastOrder'].notna()]

    # 특정 열 제거 및 이상치 처리
    df.drop(columns=['OrderAmountHikeFromlastYear'], inplace=True)
    df = df[~df['DaySinceLastOrder'].isin([31, 46, 30])]
    return df

def encode_and_scale_data(df):
    # 레이블 인코딩
    label_encoders = {
        'PreferredLoginDevice': LabelEncoder(),
        'PreferredPaymentMode': LabelEncoder(),
        'Gender': LabelEncoder(),
        'PreferedOrderCat': LabelEncoder(),
        'MaritalStatus': LabelEncoder()
    }
    for col, encoder in label_encoders.items():
        df[col] = encoder.fit_transform(df[col].astype(str))

    # 수치형 특성 스케일링
    numeric_columns = ['CityTier', 'WarehouseToHome', 'HourSpendOnApp',
                       'NumberOfDeviceRegistered', 'SatisfactionScore', 'NumberOfAddress', 'Complain',
                       'CouponUsed', 'OrderCount', 'DaySinceLastOrder', 'CashbackAmount']

    scaler = StandardScaler()
    df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

    return df

def main():
    st.title('고객 이탈 예측 앱')

    # 1. 데이터 로드
    df = load_customer_data()
    model = load_model()

    if st.button('랜덤 고객 선택'):
        # 2. 랜덤으로 하나의 행 데이터 선택
        selected_customer = df.sample(n=1).iloc[0]

        # 3. 선택된 고객 정보 출력
        st.subheader('선택된 고객 정보')

        with st.expander("고객 정보 보기"):
            for col, value in selected_customer.items():
                st.write(f'{col}: {value}')

        # 4. Churn과 CustomerID 열 제거
        customer_data = selected_customer.drop(['Churn', 'CustomerID'])

        # 5. 데이터 전처리
        preprocessed_data = preprocess_data(pd.DataFrame([customer_data]))
        encoded_data = encode_and_scale_data(preprocessed_data)

        # 6. 예측 수행
        churn_prob = model.predict_proba(encoded_data)[0][1]
        retain_prob = 1 - churn_prob

        # 7. 예측 결과 출력
        st.subheader('예측 결과')
        st.write(f'이탈 확률: {churn_prob:.2%}')
        st.write(f'유지 확률: {retain_prob:.2%}')

        # 8. 시각화
        fig = go.Figure(data=[go.Bar(x=['이탈', '유지'], y=[churn_prob, retain_prob])])
        fig.update_layout(title='이탈/유지 확률', yaxis_title='확률')
        st.plotly_chart(fig)

if __name__ == '__main__':
    main()

Overwriting app.py


In [42]:
!ls

 app.py  'E Commerce Dataset.xlsx'   node_modules   package-lock.json   url.txt
 drive	  logs.txt		     package.json   sample_data


In [43]:
!pip install streamlit joblib numpy

In [44]:
!streamlit run /content/app.py &>/content/logs.txt &

In [45]:
!npm install localtunnel
!npx localtunnel --port 8501 &>/content/url.txt &


up to date, audited 23 packages in 1s

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


In [47]:
!cat /content/url.txt

your url is: https://thin-carpets-invent.loca.lt


In [48]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))


Password/Enpoint IP for localtunnel is: 34.82.25.201
